In [3]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cmocean as cmo
import seawater as sw
import sys

sys.path.append('../')
import src.data.export_log_data as efuncs
import src.data.read_functions as rfuncs

drive=os.path.join('x:/SONAR ARCHIVE','AUV','AUV Control Centre projects and data 2020','Skate Project Mission 1-Baited_data20-10-20')
drive=os.path.join('x:/SONAR ARCHIVE','AUV','AUV Projects 2021')
campaign='Skate Project Mission Baited May 2021'
folder = 'files'
subfolder='log'
savefolder='processed-log-data'


# root dir
rootpath = os.path.dirname(os.getcwd())
# log files directory
logdir=os.path.join(drive,campaign,folder,subfolder)
# save directory
savedir=os.path.join(drive,campaign,folder,savefolder)
os.makedirs(savedir, exist_ok=True)  

In [4]:
## Umzip all the files (if not unzipped already)
## CTD data
files=glob.glob(logdir+'\*ctd*')# list files
if files:
# determine if files are zipped. If yes, unzip, if no, continue
    ctdfiles=os.path.join(logdir,'*ctd-slocum*.xml.gz')    
    import gzip, shutil
    flelist=glob.glob(ctdfiles)
    for i in flelist:
        xfle=os.path.splitext(i)[0]
        if not os.path.isfile(xfle):
            with gzip.open(i, 'r') as f_in, open( i[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

files=glob.glob(logdir+'\*sbp*')
if files:
    # determine if files are zipped. If yes, unzip, if no, continue
    files=os.path.join(logdir,'*sbp*.xml.gz')    
    import gzip, shutil
    flelist=glob.glob(files)
    for i in flelist:
        xfle=os.path.splitext(i)[0]
        if not os.path.isfile(xfle):
            with gzip.open(i, 'r') as f_in, open( i[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                
files=glob.glob(logdir+'\*gps*')
if files:    
    # determine if files are zipped. If yes, unzip, if no, continue
    files=os.path.join(logdir,'*gps-*.xml.gz')    
    import gzip, shutil
    flelist=glob.glob(files)
    for i in flelist:
        xfle=os.path.splitext(i)[0]
        if not os.path.isfile(xfle):
            with gzip.open(i, 'r') as f_in, open( i[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

files=glob.glob(logdir+'\*nav*')
if files: 
# determine if files are zipped. If yes, unzip, if no, continue
    navfiles=os.path.join(logdir,'*navigator*.xml.gz')    
    import gzip, shutil
    flelist=glob.glob(navfiles)
    for i in flelist:
        xfle=os.path.splitext(i)[0]
        if not os.path.isfile(xfle):
            with gzip.open(i, 'r') as f_in, open( i[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

files=glob.glob(logdir+'\*aanderaa*')
# determine if files are zipped. If yes, unzip, if no, continue
if files: 
    files=os.path.join(logdir,'*aanderaa*.xml.gz')    
    import gzip, shutil
    flelist=glob.glob(files)
    for i in flelist:
        xfle=os.path.splitext(i)[0]
        if not os.path.isfile(xfle):
            with gzip.open(i, 'r') as f_in, open( i[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)


files=glob.glob(logdir+'\*ctd*')
if files:
# read CTD data and convert to .csv format, output to data/ctd-files/
    ctdf=rfuncs.readctdlog(logdir,savedir)

files=glob.glob(logdir+'\*sbp*')
if files:
    # read SBP data and convert to .csv format, output to data/sbp-files/
    sbpdf=rfuncs.readsbplog(logdir,savedir)

files=glob.glob(logdir+'\*gps*')
if files:    
    # read GPS data and convert to .csv format, output to data/gps-files/
    gpdf=rfuncs.readgpslog(logdir,savedir)

files=glob.glob(logdir+'\*nav*')
if files: 
    # read nav data and convert to .csv format, output to data/nav-files/
    nvdf=rfuncs.readnavlog(logdir,savedir)

files=glob.glob(logdir+'\*aanderaa*')
if files: 
    # read aanderaa data and convert to .csv format, output to data/aanderaa-files/
    aandf=rfuncs.readaandlog(logdir,savedir)

In [ ]:
# create merged file
## always gps and nav log to mege first to new df

# round timestamp to o decimal places ready for merging
nvdf=nvdf.round({'timestamp': 0})
gpdf=gpdf.round({'timestamp': 0})

# drop unwanted variables form nav data
gpdf=gpdf.drop(columns=[
     'build-number',
     'build-tag',
     'log-level',
     'rate',
     'sbf-remote-com-port',
     'sbf-local-com-port',
     'vsp-local-com-port',
     'corrections-local-com-port',
     'vsp-local-tcp-port',
     'corrections-local-udp-port',
     'vsp-enabled',
     'corrections-enabled',
     'sbf-enabled',
     'sbf-unicast-enabled',
     'sbf-unicast-port',
     'sbf-unicast-interface',
     'log-nmea',
     'log-messages',
     'multicast-receive-enabled',
     'multicast-receive-port',
     'multicast-receive-addr',
     'multicast-receive-interface',
     'messages-received',
     'raw-logged-bytes',
     'received-telnet-bytes',
     'sent-corr-bytes',
     'sent-corr-packets',
     'sent-telnet-bytes',
     'time-since-gga',
     'time-since-sent',
     'diff-age',
     'sats',
     'stnRef',
     'cogt',
     'lat-dev',
     'lon-dev',
     'sogk',
     'sogm'])

# drop unwanted variables form nav data
nvdf=nvdf.drop(columns=[
     'magnetic-deviation',
     'pressure-warnings',
     'lat-lon-precision',
     'average-water-density',
     'sound-velocity',
     'center-actuators-on-idle',
     'dead-reckoning-orientation',
     'dead-reckoning-velocity',
     'orientation',
     'position',
     'variance',
     'velocity'])

# rename duplicat variables (t,s, sv, from control module?
nvdf=nvdf.rename(columns={"time": "time-nav",
                          "salinity": "salinity-nav",
                          "temperature": "temperature-nav",
                          "sound-velocity": "sound-velocity-nav"})

auvmerge=pd.merge(nvdf, gpdf,how='outer',on='timestamp')

# ctd
if not ctdf.empty:
    ctdf=ctdf.round({'timestamp': 0})
    auvmerge=pd.merge(auvmerge, ctdf,how='outer',on='timestamp')

# aandera 
if not aandf.empty:
    aandf=aandf.round({'timestamp': 0})
    auvmerge=pd.merge(auvmerge, aandf,how='outer',on='timestamp')

# sbp 
if not sbpdf.empty:
    print('to be done later, when we have some data')          

auvmerge.to_csv(os.path.join(savedir,'auvdata.csv')) 
list(auvmerge)